In [4]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import gensim 
import os
from gensim.models import FastText
from gensim.models.callbacks import CallbackAny2Vec
from gensim.test.utils import get_tmpfile

from sklearn.manifold import TSNE

In [5]:
class EpochSaver(CallbackAny2Vec):
    "Callback to save model after every epoch"
    def __init__(self, path_prefix):
        self.path_prefix = path_prefix
        self.epoch = 0
    
    def on_epoch_end(self, model):
        output_path = '{}_epoch{}.model'.format(self.path_prefix, self.epoch)
        print("Save model for epoch {} to {}".format(self.epoch,output_path),end='\r')
        model.save(output_path)
        self.epoch += 1


In [74]:
model_in_training = gensim.models.KeyedVectors.load('/tmp/temporary_model_epoch8800.model')

In [75]:
model_in_training.wv.most_similar(["vendredi"], topn=10)

[('local', 0.36730390787124634),
 ('nombreux', 0.2856573760509491),
 ('v', 0.26673901081085205),
 ('suffis', 0.25181078910827637),
 ('moyen', 0.249031201004982),
 ('gym', 0.24639834463596344),
 ('don', 0.2458961457014084),
 ('ma', 0.24297629296779633),
 ('psychosomat', 0.23385968804359436),
 ('remont', 0.23304679989814758)]